## 用于测试TPM文件中的gene_symbol列能否和pVACseq结果文件中的gene列匹配

In [1]:
import pandas as pd
import os


In [30]:
TPM_path='/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol.csv'
# tpm=pd.read_csv(TPM_path,index_col=0)
tpm=pd.read_csv(TPM_path)


In [34]:
tpm.head()

,gene_ID,gene_symbol,Pt1_Pre_AD101148-6,Pt1_On_AD174047-6,Pt10_Pre_E9047565-6,Pt10_On_E9047632-6,Pt101_Pre_AD486328-5,Pt101_On_AD681975-5,Pt103_Pre_AE134058-2,Pt103_On_AE134059-6,...,Pt9_Pre_E9021024-6,Pt9_On_E9047644-7,Pt90_Pre_AD467873-6,Pt92_Pre_AE134060-5,Pt92_On_AE148750-5,Pt93_On_AE070988-5,Pt94_Pre_AD732850-6,Pt94_On_AE373242-6,Pt98_Pre_AD733586-8,Pt98_On_AE086717-6
0,1,A1BG,8.181307,10.642141,3.391016,4.318424,4.331413,3.651002,12.569035,12.539079,...,3.437163,4.621245,5.973345,1.541307,3.406961,3.972505,5.064817,110.887060,4.701648,4.049321
1,10,NAT2,0.000000,0.057901,0.000000,0.115829,0.000000,0.266861,0.000000,0.000000,...,0.606337,0.658621,0.000000,0.000000,0.000000,0.000000,0.055708,29.781233,0.000000,0.000000
2,100,ADA,27.868108,12.344321,28.546798,36.194928,66.517490,78.037661,31.422172,6.781503,...,19.749346,20.485799,29.902344,37.049580,52.096923,5.007066,24.663177,11.180801,9.424623,18.592148
3,1000,CDH2,1.131863,0.119337,3.372158,11.049641,3.186713,7.019172,0.231551,0.051235,...,98.268472,121.799328,4.111657,9.877695,7.786109,0.762186,2.509534,26.071303,2.892955,2.212811
4,10000,AKT3,70.242766,106.608887,132.687164,157.627160,21.163046,25.166711,8.201503,58.732377,...,94.081602,90.423705,31.270480,58.328645,45.222772,76.565628,49.500965,25.485694,53.206551,66.910813


In [33]:
#tpm第一列重命名为gene_ID
tpm.rename(columns={tpm.columns[0]:'gene_ID'}, inplace=True)

In [37]:
#gene_symbol为NA的gene_ID
na_gene_ID=tpm[tpm['gene_symbol'].isna()]['gene_ID'].tolist()

In [40]:
len(na_gene_ID)

119

In [42]:
#另存为txt文件
with open('/work/longyh/BY/processed/na_gene_ID.txt','w') as f:
    for gene_id in na_gene_ID:
        f.write(str(gene_id) + '\n')

手动查看未匹配的gene ID

In [43]:
# pip install mygene  # 若环境还没有
import mygene

mg = mygene.MyGeneInfo()


In [73]:

entrez_id = "100128573"  # 例：TP53
res = mg.getgene(entrez_id, fields="symbol,name,entrezgene,taxid")
if res is not None:
    print(res["symbol"]+'| '+res["name"]) 
else:
    print(f"No data found for Entrez ID {entrez_id}")
# 若想更稳妥，可用 getgenes 批量：
# res_list = mg.getgenes(["7157", "1956"], fields="symbol,name,entrezgene,taxid")

Input sequence provided is already in string format. No operation performed


No data found for Entrez ID 100128573


In [72]:
from pathlib import Path
import pandas as pd
import re
import mygene

in_path = Path("/work/longyh/BY/processed/na_gene_ID.txt")
out_path = Path("/work/longyh/BY/processed/na_gene_ID_symbol.csv")
unmatched_path = Path("/work/longyh/BY/processed/na_gene_ID_unmatched.txt")  # 可选

# 读取并规范化 ID（去除空行；将形如 "7157.0" 转为 "7157"）
lines = in_path.read_text(encoding="utf-8").splitlines()
ids_raw = [ln.strip() for ln in lines if ln.strip()]

def normalize_id(s: str) -> str:
    s = str(s).strip()
    if re.fullmatch(r"\d+(?:\.0)?", s):
        return s.split(".")[0]
    return s

ids = [normalize_id(x) for x in ids_raw]

mg = mygene.MyGeneInfo()

# 关键：scopes 增加 "retired"，fields 加上 name 兜底，species 保持 human
res = mg.querymany(ids,
                   scopes="entrezgene,retired",
                   fields="symbol,name,taxid",
                   species="human")

# 按原始 query 组装，symbol 缺失时用 name 兜底
by_query = {}
for r in res:
    q = str(r.get("query"))
    if r.get("notfound"):
        by_query[q] = pd.NA
    else:
        sym = r.get("symbol")
        if not isinstance(sym, str) or sym.strip() == "":
            sym = r.get("name")  # 兜底：用 name
        by_query[q] = sym if isinstance(sym, str) else pd.NA

df = pd.DataFrame({"entrez_id": ids})
df["symbol"] = [by_query.get(i, pd.NA) for i in df["entrez_id"]]

df.to_csv(out_path, index=False)
print(f"Saved: {out_path} ({len(df)} rows)")
print(df.head())

# 可选：另存未匹配列表
unmatched = df[df["symbol"].isna()]["entrez_id"].drop_duplicates()
if len(unmatched) > 0:
    unmatched.to_csv(unmatched_path, index=False, header=["entrez_id"])
    print(f"Unmatched IDs saved to: {unmatched_path} ({len(unmatched)} unique)")

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
23 input query terms found no hit:	['100128573', '100130100', '100272216', '100500815', '100500833', '100500892', '100616209', '1006164


Saved: /work/longyh/BY/processed/na_gene_ID_symbol.csv (119 rows)
   entrez_id     symbol
0  100127889     CC2D2B
1  100128338     IQANK1
2  100128573       <NA>
3  100130000  PDE4DIPP6
4  100130093     SNAP47
Unmatched IDs saved to: /work/longyh/BY/processed/na_gene_ID_unmatched.txt (23 unique)


手动查询网站，补充gene symbol

In [74]:
tpm_path = Path("/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol.csv")
manual_map_path = Path("/work/longyh/BY/processed/na_gene_ID_symbol.csv")
out_tpm_path = Path("/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol_updated.csv")

In [75]:
tpm=pd.read_csv(tpm_path)
manual_map=pd.read_csv(manual_map_path)

In [76]:
print(manual_map.head())

   entrez_id        symbol
0  100127889        CC2D2B
1  100128338        IQANK1
2  100128573  LOC100128573
3  100130000     PDE4DIPP6
4  100130093        SNAP47


In [77]:
#如果manual_map中symbol不为NA，则更新tpm中的gene_symbol
for index, row in manual_map.iterrows():
    entrez_id = row['entrez_id']
    symbol = row['symbol']
    if pd.notna(symbol):
        tpm.loc[tpm['gene_ID'] == entrez_id, 'gene_symbol'] = symbol

In [79]:
#查看更新后的tpm
tpm['gene_symbol'].isna().sum()

0

In [80]:
#另存
tpm.to_csv(out_tpm_path, index=False)

测试实际匹配情况，以Pt3为例

In [3]:
Pt3_path='/work/longyh/BY/processed/naive/Pt3_tumor.MHC_I.all_epitopes.aggregated.tsv'
pt3=pd.read_csv(Pt3_path,sep='\t')

In [4]:
pt3.columns

Index(['ID', 'Index', 'A*01:01', 'A*23:01', 'B*39:01', 'B*44:03', 'C*04:01',
       'C*12:03', 'Gene', 'AA Change', 'Num Passing Transcripts',
       'Best Peptide', 'Best Transcript', 'MANE Select', 'Canonical', 'TSL',
       'Allele', 'Pos', 'Prob Pos', 'Num Included Peptides',
       'Num Passing Peptides', 'IC50 MT', 'IC50 WT', '%ile MT', '%ile WT',
       'RNA Expr', 'RNA VAF', 'Allele Expr', 'RNA Depth', 'DNA VAF', 'Tier',
       'Evaluation'],
      dtype='object')

In [6]:
print(pt3.shape)
print(tpm.shape)

(184, 32)
(22187, 110)


In [10]:
#pt3的Gene列有多少能在tpm的gene_symbol中找到
matched_genes=pt3['Gene'].isin(tpm['gene_symbol'])
print(f"Number of matched genes: {matched_genes.sum()}")

Number of matched genes: 176


统计所有的naive中基因名匹配情况

In [ ]:
import pandas as pd
from pathlib import Path

tpm_path = Path("/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol_updated.csv")
naive_dir = Path("/work/longyh/BY/processed/naive")
out_dir = naive_dir / "unmatched_genes"
out_dir.mkdir(exist_ok=True)

tpm = pd.read_csv(tpm_path, index_col=0)
gene_set = set(tpm["gene_symbol"].dropna().astype(str))

summary = []
all_unmatched = []  # 汇总所有文件的未匹配基因（带来源文件）

for f in sorted(naive_dir.glob("*")):
    if not f.is_file():
        continue
    if f.suffix.lower() not in {".tsv", ".csv"}:
        continue

    sep = "\t" if f.suffix.lower() == ".tsv" else ","
    df = pd.read_csv(f, sep=sep)

    if "Gene" not in df.columns:
        continue

    genes = df["Gene"].dropna().astype(str)
    matched = genes.isin(gene_set)
    unmatched_values = genes[~matched].unique()

    # 保存当前文件的未匹配基因到单独文件
    if len(unmatched_values) > 0:
        out_file = out_dir / f"{f.stem}_unmatched_genes.txt"
        pd.Series(unmatched_values).to_csv(out_file, index=False, header=["Gene"])
        all_unmatched.extend([(f.name, g) for g in unmatched_values])

    summary.append({
        "file": f.name,
        "matched_genes": int(matched.sum()),
        "unmatched_genes": int(len(unmatched_values)),
        "total_genes": int(len(genes)),
        "match_ratio": matched.mean(),
        "unique_genes_in_file": int(genes.nunique()),
    })

# 汇总表（匹配统计）
result = pd.DataFrame(summary).sort_values("match_ratio", ascending=False)
print(result)

# 可选：汇总所有未匹配基因及来源文件，便于手动检查
if all_unmatched:
    unmatched_df = pd.DataFrame(all_unmatched, columns=["file", "Gene"])
    unmatched_df.to_csv(out_dir / "all_unmatched_genes_with_source.csv", index=False)
    # 如需去重只留基因名：unmatched_df["Gene"].drop_duplicates().to_csv(out_dir / "all_unmatched_unique_genes.txt", index=False)

In [82]:
result['match_ratio'].value_counts(bins=10).sort_index()

(0.914, 0.923]    1
(0.923, 0.932]    0
(0.932, 0.94]     0
(0.94, 0.949]     1
(0.949, 0.957]    2
(0.957, 0.966]    6
(0.966, 0.974]    9
(0.974, 0.983]    6
(0.983, 0.991]    2
(0.991, 1.0]      6
Name: count, dtype: int64

## 得到RNA seq原始SRA与病人ID的配对

In [2]:
mapping_file='/work/longyh/BY/processed/RNA-seq_SRA_Pt.csv'
mapping=pd.read_csv(mapping_file)

In [4]:
mapping.head()

,accession,sample_accession,sample_alias,sample_title
0,SRR5088813,SRS1844529,GSM2420259,Pt1_Pre_AD101148-6
1,SRR5088814,SRS1844530,GSM2420260,Pt1_On_AD174047-6
2,SRR5088815,SRS1844531,GSM2420261,Pt2_Pre_AD101150-6
3,SRR5088816,SRS1844532,GSM2420262,Pt2_On_AD174046-6
4,SRR5088817,SRS1844533,GSM2420263,Pt23_On_AD486568-8


In [5]:
#mapping中sample_title去掉含有_Pre_的行
mapping_filtered=mapping[~mapping['sample_title'].str.contains('_Pre_')]

In [8]:
#mapping中新建一列Patient_ID，从sample_title中提取Pt编号（_前的部分）,使用.loc
mapping_filtered.loc[:, 'Patient_ID'] = mapping_filtered['sample_title'].str.split('_').str[0]

In [12]:
mapping_filtered.shape

(58, 5)

In [13]:
mapping_filtered.head()

,accession,sample_accession,sample_alias,sample_title,Patient_ID
1,SRR5088814,SRS1844530,GSM2420260,Pt1_On_AD174047-6,Pt1
3,SRR5088816,SRS1844532,GSM2420262,Pt2_On_AD174046-6,Pt2
4,SRR5088817,SRS1844533,GSM2420263,Pt23_On_AD486568-8,Pt23
7,SRR5088820,SRS1844536,GSM2420266,Pt58_On_AE045812-10,Pt58
10,SRR5088823,SRS1844539,GSM2420269,Pt82_On_AD125218-6,Pt82


In [3]:
#加载naive队列
naive=pd.read_excel('/work/longyh/BY/processed/merged_WES_meta_cohort_naive.xlsx')
naive.columns

Index(['Patient', 'Cohort', 'Run'], dtype='object')

In [15]:
#mapping_filtered保留Patient_ID存在于naive Patient中的行
mapping_filtered_naive = mapping_filtered[mapping_filtered['Patient_ID'].isin(naive['Patient'])]

In [16]:
mapping_filtered_naive.to_csv('/work/longyh/BY/processed/RNA-seq_naive.csv', index=False)

In [17]:
mapping_filtered_naive['accession'].to_csv('/work/longyh/BY/processed/RNA-seq_naive_accession.txt', index=False, header=False)